In [1]:
# import packages
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# import dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
# prepare the train and test set
train_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
train_set = np.array(train_set,dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [4]:
# get the total number of users and movies in the database
nb_users = int(max(max(train_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(train_set[:,1]), max(test_set[:,1])))

In [5]:
# convert the data into an array with users in rows and movies in coulmums 

def convert(data):
    new_data = []
    for id_users in range(1, nb_users +1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
train_set = convert(train_set)
test_set = convert(test_set)

In [6]:
# convert the data into torch tensor
train_set = torch.FloatTensor(train_set)
test_set = torch.FloatTensor(test_set)

In [7]:
# create the architecture of the stacked autoencoder neural network
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self,x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
stacked_autoencoder = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(stacked_autoencoder.parameters(), lr=0.01, weight_decay = 0.5)

nb_epoch = 256
for epoch  in range(1, nb_epoch + 1):
    train_loss = 0
    # s = 0.
    for id_user in range(nb_users):
        input_val = Variable(train_set[id_user].unsqueeze(0))
        target = input_val.clone()
        if torch.sum(target.data > 0) > 0:
            output = stacked_autoencoder(input_val)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target) 
            # mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += loss.item()
            # s += 1.
            optimizer.step()
    print('epoch:' +str(epoch)+ 'loss:' + str(train_loss))

test_loss = 0
s = 0.

for id_users in range(nb_users):
    input_val = Variable(train_set[id_users]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = stacked_autoencoder(input_val)
        target.requir_grad = False
        output[target == 0] = 0
        loss = criterion(output,target)
        # mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += loss.item()
        # s += 1.
print('test_loss:' + str(test_loss))

epoch:1loss:150.94560038065538
epoch:2loss:58.26462555862963
epoch:3loss:53.457291351864114
epoch:4loss:51.76683685788885
epoch:5loss:50.96671332651749
epoch:6loss:50.495222518453375
epoch:7loss:50.185861349571496
epoch:8loss:50.00222263089381
epoch:9loss:49.87872382882051
epoch:10loss:49.76085396425333
epoch:11loss:49.687861835816875
epoch:12loss:49.63344586559106
epoch:13loss:49.60119485144969
epoch:14loss:49.55087251320947
epoch:15loss:49.55050618969835
epoch:16loss:49.485379446763545
epoch:17loss:49.481970206368715
epoch:18loss:49.430195577209815
epoch:19loss:49.43802142387722
epoch:20loss:49.40070888772607
epoch:21loss:49.41402411973104
epoch:22loss:49.38448932021856
epoch:23loss:49.388632896123454
epoch:24loss:49.35826215078123
epoch:25loss:49.3688422201667
epoch:26loss:49.34111655084416
epoch:27loss:49.33578240475617
epoch:28loss:49.30102851532865
epoch:29loss:48.97761042509228
epoch:30loss:48.85038861411158
epoch:31loss:48.37506943929475
epoch:32loss:48.492023854050785
epoch:33